In [1]:
import requests
import time
#from time import strftime
import datetime
import numpy as np
import re
import xmltodict
import pprint# as pp
#import process_command
#print(dir(process_command))
import ast

In [2]:
def openWebsite(adress):
    r = requests.get(adress)
    return r.text

In [3]:
def prettyPrint(text):
    pp = pprint.PrettyPrinter(indent=1)
    pp.pprint(text)

In [4]:
def process_command(command, param):  
    beforeId=""
    for c in command:
        beforeId+= "/"+c
    behindId=""
    for key in param.keys():
        behindId+= "&"+str(key)+"="+str(param[key])
    return {"beforeId":beforeId, "behindId":behindId}

In [5]:
def createAdress(command,param):
    baseurl="https://api.tfl.gov.uk"
    commands = process_command(command,param)
    xmlFormatter=""#&forceXml=true"
    accessId ="app_id=1f2bf917&app_key=985d619c2276f4318dff45f31f1274fa"
    adress = baseurl + commands["beforeId"] + "?" + accessId + commands["behindId"] + xmlFormatter
    print(adress)
    return adress
# https://api.tfl.gov.uk/journey/journeyresults/westminster/to/
# bank?app_id=1f2bf917&app_key=985d619c2276f4318dff45f31f1274fa&forceXml=true

In [6]:
#addr=createAdress(['journey','journeyresults','westminster','to','bank'], {})
#cont=openWebsite(addr)
#print(type(cont))
#myDict=ast.literal_eval(cont)
#print(type(myDict))

In [7]:
def responseToDict(response):
    return ast.literal_eval(response)

In [8]:
def getData(command, param):
    response_xml = openWebsite(createAdress(command, param))
    return responseToDict(response_xml)    

In [9]:
#a = getData(['StopPoint/Search',], {'query':'AldgateEast'})

#for i in range(20):
#    print(a[i]['lineName'])
#print(a['matches'][0]['name'])
#print(a['matches'][0]['id'])


In [10]:
#a = getData(['StopPoint/940GZZLUADE/arrivals',], {})

In [11]:
def nameToId(name):
    response = getData(['StopPoint/Search',], {'query':name})
    return response['matches'][0]['id']

In [12]:
#nameToId("AldgateEast")

In [13]:
def short_departureAt(name):
    station_id=nameToId(name)
    command, param = ['StopPoint',str(station_id),'arrivals'], {}
    response = getData(command, param)
    return response

In [14]:
#prettyPrint(short_departureAt("BakerStreet"))

In [15]:
def nice_nextXDeparturesAt(name):
    response = short_departureAt(name)
    print("Next departures at "+str(response[0]['stationName'])+":\n")
    print("   "+"Line".ljust(23)+"Destination".ljust(30)+"Time")#                  (intern data)")
    print("   "+"----".ljust(23)+"-----------".ljust(30)+"----")
    for ind,dep in enumerate(response):
        print("   "+dep['lineName'].ljust(23)+dep['towards'].ljust(30)[0:30]+dep['expectedArrival'][11:19])

In [16]:
#nice_nextXDeparturesAt("BakerStreet")

In [36]:
def convertNextDepDictToArray(myDict):
    length=6
    arr = [["" for j in range(length)] for i in range(len(myDict))]
    #arr = np.chararray((len(myDict, length),itemsize=100,unicode=True)

    for i in range(len(myDict)):
        arr[i][0] = str(myDict[i]['expectedArrival'])
        arr[i][1] = myDict[i]['lineName']
        arr[i][2] = myDict[i]['towards']
    return arr

In [37]:
def getNextDepartures(arr):
    now=datetime.datetime.now() - datetime.timedelta(hours=2)
    arr_new = arr#[:]+""
    for i in range(len(arr_new)):
        formatX="%Y-%m-%dT%H:%M:%Sz"
        arr_new[i][3]=datetime.datetime.strptime(arr_new[i][0], formatX)
        arr_new[i][4]=(arr_new[i][3]-now).total_seconds() 
    return arr_new

In [38]:
def sortArray(arr, col=5):
    for i in range(len(arr)):
        for j in range(len(arr)-i-1):
            if float(arr[i][col])>float(arr[i+j+1][col]):
                #arr[i], arr[i+j+1] = arr[i+j+1], arr[i]
                arr[[i, i+j+1]] = arr[[i+j+1, i]]
    return arr

In [39]:
def addNoise(arr, maxNoise=60):
    noise = (60*np.random.rand(len(arr))-30).astype(int)
    orig = np.array(arr)[:,4].astype(int)
    new=noise+orig
    arr=np.array(arr)
    arr[:,5] = new
    return arr

In [40]:
response=short_departureAt("BakerStreet")
arr = convertNextDepDictToArray(response)

https://api.tfl.gov.uk/StopPoint/Search?app_id=1f2bf917&app_key=985d619c2276f4318dff45f31f1274fa&query=BakerStreet
https://api.tfl.gov.uk/StopPoint/940GZZLUBST/arrivals?app_id=1f2bf917&app_key=985d619c2276f4318dff45f31f1274fa


In [41]:
arr_new = getNextDepartures(arr)
sort = addNoise(arr_new)
sort = sortArray(arr_new, col=5)
for i in range(len(sort)):
    #print(str(int(sort[i][4]/60))+":"+str(int(sort[i][4]%60)),sort[i,5],"\t",sort[i][1],"\t to",sort[i][2])
    print(int(sort[i,5]),"\t",sort[i][1],"\t to",sort[i][2])

ValueError: could not convert string to float: ''

In [42]:
class timer:
    import timer
    def __init__(self,tickLength=60,debug=False):
        self.span = tickLength
        self.debug=debug
        self.goTick=False

    def tick(self, debug=False):
        now=time.time()
        if not self.goTick:
            self.starttime = now+0.
            self.goTick=True
            self.lastTick = now+0.
        sleepTime = span - ((now - starttime) % span)
        nextTick = now + sleepTime
        if nextTick > self.lastTick + self.span*1.1:
            if self.debug:
                print("  - timer: missed a tick")
        if self.debug:
            print("  - timer: sleepTime = "+str(sleepTime))
        time.sleep(sleepTime)    
        self.lastTick = time.time()
    
    def tickList_set(self,aList, epsilon = 0.1):
        self.tickList = aList
        self.tickListPointer=0
        self.epsilon=epsilon
    
    def tickList_go(self): # use this method in a loop before doing something! First call starts counter.
        if self.tickListPointer==0:
            self.tickList_startTime=time.time()
        differenceReal = time.time() - self.tickList_startTime
        differencePlanned = self.tickList[self.tickListPointer]
            
        if differenceReal < differencePlanned:
            sleepTime = differencePlanned-differenceReal 
            if self.debug:
                print("  - timer: sleepTime =",sleepTime)
            time.sleep(sleepTime)
        elif not self.tickListPointer==0:
            if self.debug:
                print("  - timer: warning. took to long time. Moving immidiatelly")

        if self.tickListPointer==len(self.tickList)-1:
            if self.debug:
                print("  - timer: tick list ends after next action.")
            return False
        self.tickListPointer+=1
        return True